In [87]:
class BelyiMap:
    def __init__(self,f,check=True):
        self._f = f
        return
    def __repr__(self):
        return self._f.__repr__()

In [88]:
BelyiMap(1)

1

In [1]:
def is_super_cyclotomic(f):
    """
    This is quite close to `is_cyclotomic_product` but allows for leading coefficients and powers of x
    """
    x = f.parent().gen()
    return f.degree() > 0 and all(g.is_cyclotomic() or g == x for g,_ in f.factor())

In [2]:
x = polygen(QQ)
is_super_cyclotomic(2*(x^2 + 1)*x)

True

In [34]:
class SuperCyclotomicPolynomial:
    def __init__(self,f):
        if isinstance(f, dict):
            # Assume this is a list of exponents giving x^i_1 * Phi_1(x)^i_2 * ...
            self._exponents = {i:e for i,e in f.items() if e != 0}
            x = polygen(QQ)
            self._poly = prod((x if i == 0 else cyclotomic_polynomial(i))^(e) for i,e in f.items())
            return
        if isinstance(f, list):
            # Assume this is a list of exponents giving x^i_1 * Phi_1(x)^i_2 * ...
            self.__init__(dict(enumerate(f)))
            return
        # assume it was a polynomial
        assert is_super_cyclotomic(f) and f.is_monic()
        self._poly = f
        x = f.parent().gen()
        self._exponents = {0 if g == x else g.is_cyclotomic(certificate=True): e for g,e in f.factor()}
    def __hash__(self):
        return hash(self.tuple())
    def max(self):
        return max(self._exponents.keys())
    def tuple(self):
        return tuple(self[i] for i in range(self.max() + 1))
    def exponents(self):
        return self._exponents
    
    def poly(self):
        return self._poly
    
    def __getitem__(self,i):
        return self._exponents.get(i,0)
    
    def __contains__(self,i):
        return i in self._exponents
    def __repr__(self):
        # TODO use format strings
        return " * ".join((["x" + ("^" + str(self[0]) if self._exponents[0] != 1 else "")] if 0 in self._exponents else []) + 
                          ["Phi_%s%s" % (i,"^" + str(e) if e != 1 else "") for i,e in self.exponents().items() if i != 0 and e != 0])
        

In [4]:
hash(SuperCyclotomicPolynomial(x*(x+1)*(x^2+x+1)^2))

2358394570051540768

In [157]:
hash(SuperCyclotomicPolynomial(x*(x+1)*(x^2+x+1)^2))

2358394570051540768

In [38]:
def SuperCyclotomics(n,m=None):
    x = polygen(QQ)
    for part in IntegerVectors(n=m)[0:n]:
        if len(part) != 0:
            if part[-1] == 0:
                continue
        yield prod((x if i == 0 else cyclotomic_polynomial(i))^(e) for i,e in enumerate(part))

In [26]:
all(is_super_cyclotomic(f) for f in SuperCyclotomics(90) if f != 1)

True

In [52]:
ans = set()
for m1,m2 in cartesian_product((range(6), range(5))):
    for f in SuperCyclotomics(1000, m=m1):
        for g in SuperCyclotomics(1000,m=m1):
            if f == g:
                continue
            if gcd(f,g) != 1:
                continue
            # tbh these still could be interesting?
            if f == 1 or g == 1:
                continue
            if gcd(gcd(f.exponents()),gcd(g.exponents())) != 1:
                continue
            if is_super_cyclotomic(f - g):
                c = (f-g).leading_coefficient()
                if c > 0:
                    if not (f,g) in ans:
                        ans.add((f,g))
                        print(SuperCyclotomicPolynomial(f)," - ", SuperCyclotomicPolynomial(g)," = ",c, SuperCyclotomicPolynomial((f-g)//c))

Phi_3  -  x  =  1 Phi_4
Phi_3  -  Phi_2  =  1 x^2
Phi_3  -  Phi_4  =  1 x
Phi_3  -  Phi_6  =  2 x
Phi_4  -  x  =  1 Phi_6
Phi_4  -  Phi_2  =  1 x * Phi_1
Phi_4  -  Phi_6  =  1 x
Phi_5  -  Phi_2  =  1 x^2 * Phi_3
Phi_5  -  Phi_3  =  1 x^3 * Phi_2
Phi_5  -  Phi_8  =  1 x * Phi_3
Phi_5  -  Phi_10  =  2 x * Phi_4
Phi_5  -  Phi_12  =  1 x * Phi_2^2
Phi_6  -  x  =  1 Phi_1^2
Phi_7  -  Phi_2  =  1 x^2 * Phi_5
Phi_7  -  Phi_3  =  1 x^3 * Phi_2 * Phi_4
Phi_7  -  Phi_5  =  1 x^5 * Phi_2
Phi_7  -  Phi_9  =  1 x * Phi_2^2 * Phi_6
Phi_7  -  Phi_14  =  2 x * Phi_6 * Phi_3
Phi_7  -  Phi_18  =  1 x * Phi_4 * Phi_3
Phi_8  -  Phi_2  =  1 x * Phi_1 * Phi_3
Phi_8  -  Phi_10  =  1 x * Phi_6
Phi_9  -  Phi_14  =  1 x * Phi_6 * Phi_4
Phi_10  -  Phi_6  =  1 x^3 * Phi_1
Phi_11  -  Phi_2  =  1 x^2 * Phi_3 * Phi_9
Phi_11  -  Phi_3  =  1 x^3 * Phi_2 * Phi_4 * Phi_8
Phi_11  -  Phi_5  =  1 x^5 * Phi_2 * Phi_6 * Phi_3
Phi_11  -  Phi_7  =  1 x^7 * Phi_2 * Phi_4
Phi_11  -  Phi_22  =  2 x * Phi_10 * Phi_5
Phi_12  -  Phi

Phi_2^2  -  Phi_1^2  =  4 x
Phi_3^2  -  x^2  =  1 Phi_2^2 * Phi_4
Phi_3^2  -  Phi_6^2  =  4 x * Phi_4
x * Phi_4  -  Phi_1 * Phi_3  =  1 Phi_2
x * Phi_4  -  Phi_2 * Phi_6  =  1 Phi_1
Phi_4 * Phi_3  -  x^2  =  1 Phi_5
Phi_4^2  -  x * Phi_3  =  1 Phi_10
Phi_4 * Phi_5  -  Phi_3 * Phi_8  =  1 x^2 * Phi_2^2
Phi_5^2  -  Phi_3 * Phi_7  =  1 x^3 * Phi_2^2
Phi_5^2  -  Phi_3 * Phi_9  =  1 x * Phi_2^2 * Phi_4^2
Phi_5^2  -  Phi_10^2  =  4 x * Phi_6 * Phi_4 * Phi_3
Phi_5^2  -  x * Phi_14  =  1 Phi_3 * Phi_4^3
Phi_6 * Phi_3  -  x * Phi_4  =  1 Phi_10
Phi_6 * Phi_4  -  x^2  =  1 Phi_10
Phi_6 * Phi_5  -  Phi_4 * Phi_8  =  1 x^3
Phi_6 * Phi_5  -  Phi_3 * Phi_10  =  2 x^3
Phi_6 * Phi_5  -  Phi_4 * Phi_12  =  1 x^2 * Phi_3
Phi_6^2  -  x^2  =  1 Phi_1^2 * Phi_4
Phi_3 * Phi_7  -  Phi_12 * Phi_5  =  1 x * Phi_2^4 * Phi_6
Phi_4 * Phi_7  -  Phi_8 * Phi_5  =  1 x^2 * Phi_2^2 * Phi_6
Phi_4 * Phi_7  -  Phi_3 * Phi_9  =  1 x^2 * Phi_5
Phi_4 * Phi_7  -  Phi_10 * Phi_5  =  1 x * Phi_6 * Phi_3^2
Phi_6 * Phi_7  -  Phi

Phi_4 * Phi_6^2  -  Phi_1^2 * Phi_10  =  1 x * Phi_8
Phi_6 * Phi_5^2  -  x * Phi_3 * Phi_7  =  1 Phi_4 * Phi_12 * Phi_8
Phi_2^2 * Phi_7  -  x * Phi_4 * Phi_5  =  1 Phi_3^2 * Phi_8
Phi_2^2 * Phi_7  -  Phi_3^2 * Phi_8  =  1 x * Phi_4 * Phi_5
Phi_6 * Phi_3 * Phi_7  -  Phi_4 * Phi_8 * Phi_5  =  1 x^5
Phi_4^2 * Phi_7  -  Phi_6 * Phi_5^2  =  1 x^2 * Phi_9
Phi_4^2 * Phi_7  -  x^2 * Phi_9  =  1 Phi_6 * Phi_5^2
Phi_2^2 * Phi_8  -  Phi_4^3  =  2 x * Phi_1^2 * Phi_3
Phi_3 * Phi_8 * Phi_5  -  Phi_4^2 * Phi_7  =  1 x * Phi_16
Phi_3 * Phi_8^2  -  Phi_4^2 * Phi_9  =  1 x * Phi_1^2 * Phi_2^2 * Phi_10
Phi_4^2 * Phi_8  -  x^2 * Phi_5  =  1 Phi_3 * Phi_14
Phi_4^2 * Phi_8  -  Phi_6^2 * Phi_5  =  1 x * Phi_9
Phi_4^2 * Phi_8  -  x^2 * Phi_10  =  1 Phi_6 * Phi_7
Phi_4 * Phi_8 * Phi_7  -  Phi_10 * Phi_5^2  =  1 x^5 * Phi_3
Phi_4 * Phi_8^2  -  x * Phi_6 * Phi_7  =  1 Phi_22
Phi_4 * Phi_8^2  -  Phi_6^2 * Phi_7  =  1 x * Phi_15
Phi_6 * Phi_8 * Phi_5  -  x * Phi_3 * Phi_9  =  1 Phi_12 * Phi_14
Phi_6^2 * Phi_8  - 

Some observations:
------------------

If we have an identity with all odd indices and even powers of x, then $x\mapsto -x$ has the effect of sending
$$\Phi_d \mapsto \Phi_{2d}$$
and fixes the $x^{2n}$ terms so we get an identity again with $x$ negated.

If we begin with a supercyclotomic polynomial of the form $$F_n = \prod_{d|n,d\ne 1} \Phi_d = 1 + x + \cdots + x^{n-1}$$
we get the relation
$$F_n - F_m = x^m F_{n-m}$$
and the version with the term on the right swapped with $F_m$

In [ ]:

print(factor(x^8 + x^7 + 4*x^6 + 3*x^5 + 6*x^4 + 3*x^3 + 4*x^2 + x + 1))
print(factor(x^8 + 2*x^7 + 3*x^6 + 4*x^5 + 5*x^4 + 4*x^3 + 3*x^2 + 2*x + 1))

factor(-x^7 + x^6 - x^5 + x^4 - x^3 + x^2 - x)

In [179]:
F = SuperCyclotomicPolynomial(x^8 + x^7 + 4*x^6 + 3*x^5 + 6*x^4 + 3*x^3 + 4*x^2 + x + 1)
print(F)
G = SuperCyclotomicPolynomial(x^8 + 2*x^7 + 3*x^6 + 4*x^5 + 5*x^4 + 4*x^3 + 3*x^2 + 2*x + 1)
print(G)
SuperCyclotomicPolynomial(G.poly() - F.poly())

Phi_3^1 * Phi_4^3
Phi_5^2


x^1 * Phi_14^1

In [109]:
hash(SuperCyclotomicPolynomial([2,2]))

8733642403180

In [110]:
hash(SuperCyclotomicPolynomial([2,2]))

8733642390015

In [36]:
f=SuperCyclotomicPolynomial({2:4,5:1}).poly()/(10*x*SuperCyclotomicPolynomial({4:3}).poly())

In [17]:
x = polygen(QQ)

In [37]:
f.derivative().factor()

(1/10) * x^-2 * (x - 1)^3 * (x + 1)^3 * (x^2 + 1)^-4 * (x^4 - x^2 + 1)

In [24]:
(1/((cyclotomic_polynomial(7)/(7*x*cyclotomic_polynomial(6)^4)))).derivative().factor()

(21) * (x^2 - x + 1)^3 * (x^6 + x^5 + x^4 + x^3 + x^2 + x + 1)^-2 * (x^8 + 5/3*x^7 + 1/3*x^5 + 2/3*x^4 + x^3 + 4/3*x^2 - 5/3*x + 1/3)

In [33]:
SuperCyclotomicPolynomial({2:4,5:1}).poly()

x^3 + 3*x^2 + 3*x + 1

In [50]:
(1,2) in set((1,2))

False